## 솔로랭크 랭킹에 수록된 유저들의 최근 전적을 얻기 (획득 골드량, 획득 경험치 시계열 데이터)
 - Get_match_data.ipynb 파일의 주석을 참고

In [1]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
# Get user list from rank 6 to ...
user_list = []
for i in range(1,11):
    url = 'http://www.op.gg/ranking/ladder/page={0}'.format(i)
    resp = requests.get(url)
    bs = BeautifulSoup(resp.text, 'html.parser')
    users = bs.findAll("td", {"class": "ranking-table__cell ranking-table__cell--summoner"})
    for line in users:
        line = str(line)
        line = line[line.find("<span>"):line.find("</span>")].replace("<span>","")
        user_list.append(line)
user_list[:5]

['deft123456', 'Tar2an', 'DWG BeryL', 'SKT T1 Untara', '11 14 9 7 8 20']

In [3]:
# Denote index of game player that starts crawling and ends crawling
start_number = 1 # 시작할 위치
end_number = 200

count = 1; # start number
match_number_start = 20*(start_number-1)+1 # 시작할 match number , 보통 (count-1)*20+1

In [4]:
def match_data_load(user_name, match_number_start):
    file_name1 = "user_gold_{0}.csv".format(user_name)
    file_name2 = "user_exp_{0}.csv".format(user_name)
    
    total_gold_table = pd.DataFrame()
    total_exp_table = pd.DataFrame()
    
    driver = webdriver.Chrome('/Users/김호준/Documents/chrome_webdriver/chromedriver')
    driver.implicitly_wait(3)
    driver.get('http://www.op.gg/summoner/userName={0}'.format(user_name))
    
    driver.find_element_by_xpath('//a[@data-type="soloranked"]').click()
    time.sleep(5)
    for i in range(0, 20): 
        game_detail_button = driver.find_elements_by_css_selector('a.Button.MatchDetail')
        game_detail_button[i].click()
        time.sleep(2)
        html = driver.page_source
        PlayerData = BeautifulSoup(html, 'html.parser')
        team_raw = PlayerData.find_all('tbody', {'class':'Content'})
        game_raw = PlayerData.find_all('thead', {'class':'Header'})
        
        
        win_lose = []
        champion_list = []
        name_list = []
        
        
        for line in team_raw:
            team_champion_raw = line.find_all('td', {'class':'ChampionImage Cell'})
            team_name_raw = line.find_all('td', {'class':'SummonerName Cell'})

            team_champion = []
            for champ in team_champion_raw:
                champ = champ.text.strip()
                champ = champ[:champ.find('\n')]
                team_champion.append(champ)
            champion_list.append(team_champion)

            team_name = []
            for name in team_name_raw:
                name = name.text.strip()
                team_name.append(name)
            name_list.append(team_name)
        tab_headers = driver.find_elements_by_xpath("//*[contains(text(), 'etc')]") 
        if len(tab_headers) == 0:
            game_detail_button[i].click()
            continue
        tab_headers[0].click()
        time.sleep(3)
        etc_gold = driver.find_element_by_css_selector('div.Content.tabItem.MatchDetailContent-gold')
        etc_html = etc_gold.get_attribute('innerHTML')
        script = BeautifulSoup(etc_html, 'html.parser').script
        graph_data = []
        str_script = str(script)
        offset = 0

        for idx in range(0, 20):
            start = str_script.find('"data":[', offset) + len('"data":[')
            end = str_script.find(']', start)

            graph_data.append(str_script[start:end].split(','))

            offset = end + len(']')

        table = pd.DataFrame(graph_data).add_suffix("분")
        gold_table = table[0:10]
        exp_table = table[10:]
        
        for gline in game_raw:
            game_result_raw = gline.find_all('span', {'class':'GameResult'})

            game_result = []
            for gresult in game_result_raw:
                gresult = gresult.text.strip()
                game_result.append(gresult)
            win_lose.append(game_result)
        
        win_lose = win_lose[1:]
        if win_lose[0] == ['패배']:
            win_lose = [0] * 5 + [1] * 5
        else:
            win_lose = [1] * 5 + [0] * 5

        gold_table.insert(0, "챔피언", champion_list[0] + champion_list[1])
        gold_table.insert(0, "소환사명", name_list[0] + name_list[1])
        gold_table.insert(0, "승패", win_lose)
        gold_table.insert(0, "경기번호", [i] * 10)
        exp_table.insert(0, "챔피언", champion_list[0] + champion_list[1])
        exp_table.insert(0, "소환사명", name_list[0] + name_list[1])
        exp_table.insert(0, "경기번호", [i] * 10)
        exp_table.insert(0, "승패", win_lose)

        if len(gold_table.columns) > len(total_gold_table.columns):
            gold_table_columns = gold_table.columns
            exp_table_columns = exp_table.columns
        else:
            gold_table_columns = total_gold_table.columns
            exp_table_columns = total_exp_table.columns


        total_gold_table = total_gold_table.append(gold_table)[gold_table_columns]
        total_exp_table = total_exp_table.append(exp_table)[exp_table_columns]
        game_detail_button[i].click()
        
    matchorder = match_number_start
    matchorder = matchorder + 1
    total_gold_table.to_csv(file_name1)
    total_exp_table.to_csv(file_name2)
    driver.quit()
    return matchorder

In [5]:
for user_name in user_list:
    if count == end_number:
        break
    if count < start_number:
        count += 1
        continue
    match_number_start = match_data_load(user_name, match_number_start)
    print("complete! count :",count,", error_start : count",count+1,", start_number", match_number_start)
    count += 1

WebDriverException: Message: chrome not reachable
  (Session info: chrome=63.0.3239.84)
  (Driver info: chromedriver=2.33.506120 (e3e53437346286c0bc2d2dc9aa4915ba81d9023f),platform=Windows NT 6.3.9600 x86_64)


In [46]:
# Write user_list for concatenating player data in other codes
pd.DataFrame(user_list).to_csv("user_list.csv", encoding='euc-kr')